In [1]:
import sqlite3
import os
import string

In [2]:
os.remove('TransProb_100.db')

In [3]:
con = sqlite3.connect('TransProb_100.db')

In [4]:
crsr = con.cursor()

In [5]:

'''
try:
    dele = "DROP TABLE TransProb"
    crsr.execute(dele)
except:
    print('Error')
'''

'\ntry:\n    dele = "DROP TABLE TransProb"\n    crsr.execute(dele)\nexcept:\n    print(\'Error\')\n'

In [6]:
create_table = """CREATE TABLE TransProb(
ENG_WORD VARCHAR(20) NOT NULL,
DUT_WORD VARCHAR(20) NOT NULL,
PROBABILITY FLOAT(12,10) NOT NULL,
PRIMARY KEY (ENG_WORD,DUT_WORD));
"""

cluster_index="""CREATE INDEX IX_ED_T ON TransProb (
	"ENG_WORD",
	"DUT_WORD"
);"""

DUT_INDEX ="""
        CREATE INDEX IX_DUT ON TransProb (DUT_WORD);
"""

ENG_INDEX ="""
        CREATE INDEX IX_ENG ON TransProb (ENG_WORD);
"""






In [7]:
create_table_c = """CREATE TABLE Count(
ENG_WORD VARCHAR(20) NOT NULL,
DUT_WORD VARCHAR(20) NOT NULL,
PROBABILITY FLOAT(12,10) NOT NULL,
PRIMARY KEY (ENG_WORD,DUT_WORD));
"""

cluster_index_c="""CREATE INDEX IX_ED ON Count (
	"ENG_WORD",
	"DUT_WORD"
);"""

In [8]:
crsr.execute(create_table_c)
crsr.execute(cluster_index_c)

In [9]:
crsr.execute(create_table)

In [10]:
#crsr.execute(DUT_INDEX)
#crsr.execute(ENG_INDEX)

In [11]:
crsr.execute(cluster_index)

In [12]:
import time
import os

In [13]:

UPDATED_ENGLISH_FILE = 'eng_file2.txt'
UPDATED_DUTCH_FILE = 'dutch_file2.txt'


ENG_TEST_FILE = 'eng.txt'
DUTCH_TEST_FILE = 'dutch.txt'

In [14]:
e_file = open(UPDATED_ENGLISH_FILE,'r',encoding = 'utf-8')
d_file = open(UPDATED_DUTCH_FILE,'r',encoding = 'utf-8')

In [15]:
dutch = d_file.readlines()[:10000]
eng = e_file.readlines()[:10000]

In [16]:
def remove_punc(l,lang):
    
    for i in range(len(l)):
        l[i] = l[i][:-1].lower()
        l[i] = remove_stopwords(l[i],lang)
        l[i] = remove_stuff(l[i])
    return l

def remove_stopwords(l,lang):
    eng_sw=['of', 'the', 'i', 'on', 'and', 'would', 'to', 'you', 'a', 'in', 'that', 'as', 'have', 'for', 'be', 'from', 'it', 'at', 'can', 'an', 'has', 'the', 'it', 'is', 'not', 'with', 'we', 'by', 'this', 'we', 'are', 'more', 'our', 'or', 'also', 'these', 'but', 'must']
    dut_sw=['van', 'de', 'ik', 'aan', 'en', 'zou', 'naar', 'u', 'een', 'in', 'dat', 'als', 'hebben', 'voor', 'worden', 'van', 'het', 'bij', 'kan', 'een', 'heeft', 'de', 'het', 'is', 'niet', 'met', 'wij', 'door', 'deze', 'wij', 'zijn', 'meer', 'onze', 'of', 'ook', 'deze', 'maar', 'moet']
    eng_sw = [i.lower() for i in eng_sw]
    dut_sw = [i.lower() for i in dut_sw]
    
    words = l.split()
    s = []
    
    if lang =='dutch':
        for i in words:
            if i not in dut_sw:
                s.append(i)
    elif lang =='eng':
        for i in words:
            if i not in eng_sw:
                s.append(i)
    return ' '.join(s)

def remove_stuff(l):
    #a = [ ':','.' , '\\' , '/' , ',' , ';' , '(' , ')' , '"', "\'",'1','2','3','4','5','6','7','8','9','0','?']
    for i in string.punctuation + string.digits:
        l = l.replace(i, "")
    return l

In [17]:
dutch2 = remove_punc(dutch,'dutch')
eng2 = remove_punc(eng,'eng')

In [18]:
dutch2[0]

'hervatting zitting'

In [19]:
def assign_line_no(doc,dict_lo):
    for i in range(len(doc)):
        t = doc[i].split()
        for m in t:
            dict_lo[m].add(i)
    return dict_lo

In [20]:
from collections import defaultdict

In [21]:
dutch_line_no = defaultdict(set)
eng_line_no = defaultdict(set)

In [22]:
#dutch_line_no
len(dutch_line_no)

0

In [23]:
dutch_line_no = assign_line_no(dutch2, dutch_line_no)
eng_line_no = assign_line_no(eng2, eng_line_no)

In [24]:
def initialize(foreign_no_of_words,foreign_l,english_l):
    count = 0
    init_prob = 1/foreign_no_of_words
    for i in english_l.keys():
        for j in foreign_l.keys():
            c = [i,j,init_prob]
            c1 = [i,j,0]
            try:
                crsr.execute("INSERT INTO TransProb(ENG_WORD, DUT_WORD, PROBABILITY) VALUES(?,?,?)",tuple(c))
                crsr.execute("INSERT INTO Count(ENG_WORD, DUT_WORD, PROBABILITY) VALUES(?,?,?)",tuple(c1))
            except:
                print(c)
        if count>=100:
            con.commit()
            count = 0
        print(count)
        count+=1
    con.commit()
    return True

In [25]:
b_val = initialize(len(dutch_line_no), dutch_line_no, eng_line_no)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46

29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
7